In [123]:
import pandas as pd
from pandas import *
import numpy as np
import os
from datetime import *
import statistics as stat
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import csv
from zipfile import ZipFile
import math
import collections
import filecmp
import xml.etree.ElementTree as ET
import geopandas as gpd
from shapely import wkt
import openpyxl
import re
import random

In [124]:
def read_taz(base_path):
    # START LOADING TAZ
    
    #read taz data
    psrc_taz = gpd.read_file(base_path+'/taz2010.shp').to_crs("EPSG:4326")

    #get centroid points from taz
    lat_psrc = []
    lon_psrc = []
    for polys in psrc_taz['geometry'].values:
        p = wkt.loads(str(polys))
        latlon = p.centroid.wkt
        latlon = latlon[5:].split(' ')
        lon = float(latlon[1][1:])
        lat = float(latlon[2][:-1])
        lat_psrc.append(lat)
        lon_psrc.append(lon)

    psrc_taz['lat'] = lat_psrc
    psrc_taz['lon'] = lon_psrc


    #psrc_taz['lat'] = psrc_taz['lat']/10000
    #psrc_taz['lon'] = psrc_taz['lon']/10000
    return psrc_taz
psrc_taz = read_taz(base_path)

In [125]:
def write_xml_od_walk(df, filename):
    df = df.sort_values(by='deptm')
    with open(filename, 'w') as f:
        f.write('<?xml version="1.0" encoding="UTF-8"?>\n')    
        f.write('<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n')
        template = '<person id="{id}" depart="{depart}">\n\t<walk fromTaz="{cO}" toTaz="{cD}" />\n</person>\n'
        id = 0
        for i in range(len(df)):
            demand = df.iloc[i].to_dict()
            cnt = demand['demand']
            while cnt>0:
                demand['id'] = "{}-{}_{}_{}-{}".format(demand['person_id'],int(demand['cO']), int(demand['cD']), id, int(demand['sample_index']))
                demand['cO'] = int(demand['cO'])
                demand['cD'] = int(demand['cD'])
                demand['depart'] = demand['deptm'] * 60
                f.write(template.format_map(demand))
                cnt -= 1
                id += 1
        f.write('</routes>')
        
def write_xml_od(df, filename):
    df = df.sort_values(by='deptm')
    with open(filename, 'w') as f:
        f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        f.write('<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n')
        f.write('<vType id="passenger" vClass="passenger" accel="3.2" color="yellow" decel="3.5" length="4" maxSpeed="200" lcStrategic="2000" lcSpeedGain="5" lcCooperative="1" lcSpeedGainLookahead="5" minGap="1.5" cc1="0.9" cc2="4" cc3="-8" cc4="-0.1" cc5="0.1" cc6="11.44" cc7="0.25" cc8="3.5" cc9="1.5"/>\n')
        template = '<trip type="passenger" id="{id}" depart="{depart}" fromTaz="{cO}" toTaz="{cD}" departLane="free" departSpeed="max"/>\n'
        id = 0
        for i in range(len(df)):
            demand = df.iloc[i].to_dict()
            cnt = demand['demand']
            while cnt>0:
                demand['id'] = "{}-{}_{}_{}-{}".format( demand['person_id'],int(demand['cO']), int(demand['cD']), id, int(demand['sample_index']))
                demand['cO'] = int(demand['cO'])
                demand['cD'] = int(demand['cD'])
                demand['depart'] = demand['deptm'] * 60
                f.write(template.format_map(demand))
                cnt -= 1
                id += 1
        f.write('</routes>')
        
#load the pseudo taz
def create_taz_map(base_path, psrc_taz):
    pseudo_taz = pd.read_csv(f'{base_path}//sumo_network_test//data_for_calibration//pseudo_taz.csv')


    datapath = f'{base_path}//sumo_network_test//Taz_bigger_Seattle_all_transit_with_pseudo_link.add.xml'
    tree = ET.parse(datapath)
    root = tree.getroot()
    sumo_taz = []
    for taz in root.findall('taz'):
        tazid = taz.get('id')
        sumo_taz.append(tazid)

    pseudo_loc = pseudo_taz['location'].values.T
    pseudotazs = pseudo_taz['pseudo_id'].values.T
    pseudo_mapping = {}
    for i in range(int(len(pseudo_loc)/2)):
        index_num = i*2
        loc = pseudo_loc[index_num][:-2]
        pseudo_mapping[loc] = []
        pseudo_mapping[loc].append(pseudotazs[i*2])
        pseudo_mapping[loc].append(pseudotazs[i*2+1])


    #convert sumotaz to int
    for i in range(len(sumo_taz)):
        if 'taz' not in sumo_taz[i]:
            sumo_taz[i] = int(sumo_taz[i])

    sumo_taz_dict = {k:k for k in sumo_taz}
    psrc_taz['new_taz']=psrc_taz['TAZ'].map(sumo_taz_dict)

    #patch psrc_taz
    SR99N = ((47.646869, -122.5),(47.733015,-122.336238))
    I90 = (47.646869, -122.26)

    psrc_taz.loc[(psrc_taz['new_taz'].isnull()) & (psrc_taz['COUNTY_NM']=='King') & (psrc_taz['lat']>=SR99N[0][0]) & (psrc_taz['lat']<=SR99N[1][0]) & (psrc_taz['lon']>=SR99N[0][1]) & (psrc_taz['lon']<=SR99N[1][1]),'new_taz'] = 'SR-99N'
    psrc_taz.loc[(psrc_taz['new_taz'].isnull()) & (psrc_taz['COUNTY_NM']=='King') & (psrc_taz['lat']>SR99N[1][0]) & (psrc_taz['lon']>=SR99N[0][1]) & (psrc_taz['lon']<=SR99N[1][1]),'new_taz'] = 'I-5N'
    psrc_taz.loc[(psrc_taz['new_taz'].isnull()) & (psrc_taz['COUNTY_NM']=='King') & (psrc_taz['lat']>=SR99N[0][0]) & (psrc_taz['lon']>SR99N[1][1]),'new_taz'] = 'I-5N'
    psrc_taz.loc[(psrc_taz['new_taz'].isnull()) & (psrc_taz['COUNTY_NM']=='King') & (psrc_taz['lat']<SR99N[0][0]) & (psrc_taz['lon']>=I90[1]),'new_taz'] = 'I-90'

    psrc_taz.loc[(psrc_taz['new_taz'].isnull()) & (psrc_taz['COUNTY_NM']=='King') & (psrc_taz['lat']<47.408284),'new_taz'] = 'I-5S'
    psrc_taz.loc[(psrc_taz['new_taz'].isnull()) & (psrc_taz['COUNTY_NM']=='King') & (psrc_taz['lat']<47.60) & (psrc_taz['lon']<-122.329139),'new_taz'] = 'SR-99S'
    psrc_taz.loc[(psrc_taz['new_taz'].isnull()) & (psrc_taz['COUNTY_NM']=='King') & (psrc_taz['lat']<47.60) & (psrc_taz['lon']>=-122.329139),'new_taz'] = 'I-5S'
    psrc_taz.loc[(psrc_taz['new_taz'].isnull()) & (psrc_taz['COUNTY_NM']=='King') & (psrc_taz['lat']>=47.60) & (psrc_taz['lon']>=-122.34),'new_taz'] = 'East'

    psrc_taz.loc[(psrc_taz['COUNTY_NM']=='Snohomish'),'new_taz'] = 'I-5N'
    psrc_taz.loc[(psrc_taz['COUNTY_NM']=='Pierce'),'new_taz'] = 'I-5S'

    psrc_taz.loc[(psrc_taz['TAZ'].isin([444, 445])), 'new_taz'] =446
    psrc_taz.loc[(psrc_taz['TAZ'].isin([346,347,348, 349,350,351,352, 353, 389, 390,391, 392, 393, 394, 395, 396, 397, 398])), 'new_taz'] =442
    psrc_taz.loc[(psrc_taz['TAZ'].isin([354, 355, 356, 385, 386, 387, 388, 399, 400])), 'new_taz'] =440
    psrc_taz.loc[(psrc_taz['TAZ'].isin([401, 402])), 'new_taz'] =436
    psrc_taz.loc[(psrc_taz['TAZ'].isin([357, 358, 359, 384, 403])), 'new_taz'] =434
    psrc_taz.loc[(psrc_taz['TAZ'].isin([405])), 'new_taz'] =432
    psrc_taz.loc[(psrc_taz['TAZ'].isin([407])), 'new_taz'] =427
    psrc_taz.loc[(psrc_taz['TAZ'].isin([408])), 'new_taz'] =426



    psrc_taz.loc[(psrc_taz['TAZ'].isin([633])), 'new_taz'] =610
    psrc_taz.loc[(psrc_taz['TAZ'].isin([654, 655, 673])), 'new_taz'] =653
    psrc_taz.loc[(psrc_taz['TAZ'].isin([656])), 'new_taz'] =652
    psrc_taz.loc[(psrc_taz['TAZ'].isin([657])), 'new_taz'] =650


    taz_map = psrc_taz[(~psrc_taz['new_taz'].isnull())][['TAZ', 'new_taz']].set_index('TAZ').to_dict()['new_taz']

    #tazs = taz_map.keys()
    return pseudo_mapping, taz_map

In [126]:
def apply_taz_convertion(psrc_ods, pseudo_mapping, taz_map):
    tazs = taz_map.keys()
    psrc_ods_converted = psrc_ods[psrc_ods['otaz'].isin(tazs)]
    psrc_ods_converted = psrc_ods_converted[psrc_ods_converted['dtaz'].isin(tazs)]

    #convert od files
    ori = psrc_ods_converted['otaz'].values.T
    dest = psrc_ods_converted['dtaz'].values.T

    convert_ori = []
    convert_dest = []
    convert_taz = list(tazs)


    for i in range(len(ori)):
        if ori[i] in convert_taz:
            mapped_dir = taz_map[ori[i]]
            if type(mapped_dir) == int:
                convert_ori.append(mapped_dir)
            else:
                convert_ori.append(pseudo_mapping[mapped_dir][0])
        else:
            convert_ori.append(ori[i])

        if dest[i] in convert_taz:
            mapped_dir = taz_map[dest[i]]
            if type(mapped_dir) == int:
                convert_dest.append(mapped_dir)
            else:
                convert_dest.append(pseudo_mapping[mapped_dir][1])
        else:
            convert_dest.append(dest[i])

    psrc_ods_converted['cO'] = convert_ori
    psrc_ods_converted['cD'] = convert_dest
    
    # Patch for TAZs near the border
    p = psrc_ods_converted
    north = [426, 427, 432, 434, 436, 440, 442, 446]
    north_outside = [5000, 5001, 5010, 5011, 5004, 5005]
    south = [610, 652, 650, 653]
    south_outside = [5002, 5003, 5008, 5009]

    east_outside=[5004, 5005]
    east_excluded=[604, 605, 608, 636, 627, 629, 615, 430, 432, 434, 436, 440]

    east_local = [5006, 5008]
    east_local_excluded=[604, 605, 608, 636, 627, 629, 615]

    p.loc[(p['cO'].isin(north_outside)) & (p['cD'].isin(north)) & (p['dtaz']!=p['cD']), 'demand'] = 0
    p.loc[(p['cD'].isin(north_outside)) & (p['cO'].isin(north)) & (p['otaz']!=p['cO']), 'demand'] = 0
    p.loc[(p['cO'].isin(south_outside)) & (p['cD'].isin(south)) & (p['dtaz']!=p['cD']), 'demand'] = 0
    p.loc[(p['cD'].isin(south_outside)) & (p['cO'].isin(south)) & (p['otaz']!=p['cO']), 'demand'] = 0

    p.loc[(p['cO'].isin(east_outside)) & (p['cD'].isin(east_excluded)), 'demand'] = 0
    p.loc[(p['cD'].isin(east_outside)) & (p['cO'].isin(east_excluded)), 'demand'] = 0
    p.loc[(p['cO'].isin(east_local)) & (p['cD'].isin(east_local_excluded)), 'demand'] = 0
    p.loc[(p['cD'].isin(east_local)) & (p['cO'].isin(east_local_excluded)), 'demand'] = 0

    p.loc[(p['cO'] == 5000) & (p['cD'] >=430) & (p['cD'] <=443), 'demand'] = 0
    p.loc[(p['cD'] == 5001) & (p['cO'] >=430) & (p['cO'] <=443), 'demand'] = 0
    
    p.loc[(p['cO'].isin(north_outside)) & (p['cD'].isin(north_outside)), 'demand'] = 0
    p.loc[(p['cO'].isin(south_outside)) & (p['cD'].isin(south_outside)), 'demand'] = 0
    p.loc[(p['cO'].isin(east_outside)) & (p['cD'].isin(east_outside)), 'demand'] = 0
    p.loc[(p['cO'].isin(east_local)) & (p['cD'].isin(east_local)), 'demand'] = 0
    p.loc[(p['cO'].isin(south_outside)) & (p['cD'].isin(east_outside)), 'demand'] = 0
    p.loc[(p['cO'].isin(east_outside)) & (p['cD'].isin(south_outside)), 'demand'] = 0
    #p.loc[(p['cD'] >=5000) & (p['cO'] >=5000), 'demand'] = 0
    p = p[p['demand']>0]

    return p

In [107]:
pseudo_mapping

{'I-5N': [5000, 5001],
 'I-5S': [5002, 5003],
 'I-90': [5004, 5005],
 'East': [5006, 5007],
 'SR-99S': [5008, 5009],
 'SR-99N': [5010, 5011]}

In [108]:
os.listdir(sample_dt_path)

['Bigger_Seattle.sumocfg',
 'bus_link_route.rou.xml',
 'data_extract.py',
 'Edgetest_m.xml',
 'edge_data.add.xml',
 'generate_ped_od.py',
 'ped_od.rou.xml',
 'ped_od_app_sim.csv',
 'ped_od_app_sim.txt',
 'signal_additional_revise.add.xml',
 'test_2_traffic_sig_revise_5.net.xml',
 'tripinfo_output.xml',
 'vehroute_output.xml',
 'veh_od_downtown_10pct.rou.xml',
 '__pycache__',
 'Taz_bigger_Seattle_all_fordtagg.add.xml',
 'missing_taz_edges',
 'test_2_traffic_sig_revise_6.net.xml',
 'Taz_bigger_Seattle_all_ped_transit_5.add.xml',
 'Taz_bigger_Seattle_all_transit_with_pseudo_link_2_5.add.xml',
 'signal_additional_revise_5.add.xml',
 'previous_net_no_taz_revised',
 'fcd_output.xml',
 'NULL',
 'fcd_output_5.xml',
 'fcd_output_5_30sec.xml',
 'fcd_output_obs.csv',
 'fcd_data_5.csv',
 'test_taz_Seattle.prj',
 'test_taz_Seattle.qpj',
 'test_taz_Seattle.cpg',
 'test_taz_Seattle.dbf',
 'test_taz_Seattle.shx',
 'test_taz_Seattle.shp',
 'test_taz.shx',
 'test_taz.dbf',
 'test_taz.shp',
 'test_taz.pr

In [131]:
base_path = 'C:\\Users\\Yiran\\Desktop\\sumo_calibration\\'

sample_dt_path = 'G:\\My Drive\\2021\\Bias\\SUMO_simulation\\appsim\\'
#sample_csv = pd.read_csv(sample_dt_path+'sample_trip_all_10per.csv', index_col=0)
sample_csv = pd.read_csv(sample_dt_path+'sample_trip_ht_10per.csv', index_col=0)
sample_csv.reset_index(inplace=True)
sample_csv = sample_csv.sort_values(by=['person_id', 'deptm'])
sample_csv.head(3)

# mode_d: 1: walk, 2: car, 3: ignore

test = sample_csv.copy()
test['demand'] = 1
pseudo_mapping, taz_map = create_taz_map(base_path, psrc_taz)
test2 = apply_taz_convertion(test, pseudo_mapping, taz_map)

test2.columns

Index(['level_0', 'index', 'arrtm', 'dadtyp', 'day', 'deptm', 'dorp', 'dpcl',
       'dpurp', 'dtaz', 'endacttm', 'half', 'hhno', 'id', 'mode', 'oadtyp',
       'opcl', 'opurp', 'otaz', 'pathtype', 'pno', 'sov_ff_time', 'tour',
       'tour_id', 'travcost', 'travdist', 'travtime', 'trexpfac', 'tseg',
       'tsvid', 'vot', 'person_id', 'de_hr', 'de_min', 'p_ht', 'sample_index',
       'mode_d', 'demand', 'cO', 'cD'],
      dtype='object')

In [132]:
#sample_car = test2[['person_id', 'deptm', 'arrtm', 'otaz', 'dtaz', 'de_hr', 'de_min', 'mode_d']][sample_csv['mode_d']==2]

#test = sample_car[['person_id', 'deptm', 'otaz', 'dtaz']].copy()
#write_xml_od(test2[['person_id', 'deptm', 'cO', 'cD', 'demand', 'sample_index']][test['mode_d']==2], f'{sample_dt_path}\\sample_car.rou.xml')
#write_xml_od_walk(test2[['person_id', 'deptm', 'cO', 'cD', 'demand', 'sample_index']][test['mode_d']==1], f'{sample_dt_path}\\sample_walk.rou.xml')

write_xml_od(test2[['person_id', 'deptm', 'cO', 'cD', 'demand', 'sample_index']][test['mode_d']==2], f'{sample_dt_path}\\sample_car_ht.rou.xml')
write_xml_od_walk(test2[['person_id', 'deptm', 'cO', 'cD', 'demand', 'sample_index']][test['mode_d']==1], f'{sample_dt_path}\\sample_walk_ht.rou.xml')


C:\Users\Yiran\AppData\Local\Temp\ipykernel_15608\3354925624.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  write_xml_od(test2[['person_id', 'deptm', 'cO', 'cD', 'demand', 'sample_index']][test['mode_d']==2], f'{sample_dt_path}\\sample_car_ht.rou.xml')
C:\Users\Yiran\AppData\Local\Temp\ipykernel_15608\3354925624.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  write_xml_od_walk(test2[['person_id', 'deptm', 'cO', 'cD', 'demand', 'sample_index']][test['mode_d']==1], f'{sample_dt_path}\\sample_walk_ht.rou.xml')


In [3]:
selected_taz_od = pd.read_csv(f'{base_path}\\veh_od.csv')
downtown_tazs = np.unique(selected_taz_od['O'])
del(selected_taz_od)

In [40]:
def read_trips(trips_path):
    trips_file="{}\\_trip.tsv".format(trips_path)
    df = pd.read_csv(trips_file, header = 0, sep='\t')
    return df

def get_vols(od_tazs):
    grouped = od_tazs.groupby(['otaz', 'dtaz', 'deptm', 'arrtm']).count()[['id']].rename(columns={"id":"all"})
    grouped_hov2_driver = od_tazs[(od_tazs['mode']==4) & (od_tazs['dorp']==1)].groupby(['otaz', 'dtaz', 'deptm', 'arrtm']).count()[['id']].rename(columns={"id":"hov2d"})
    grouped_hov2_passenger = od_tazs[(od_tazs['mode']==4) & (od_tazs['dorp']==2)].groupby(['otaz', 'dtaz', 'deptm', 'arrtm']).count()[['id']].rename(columns={"id":"hov2p"})
    grouped_hov3_driver = od_tazs[(od_tazs['mode']==5) & (od_tazs['dorp']==1)].groupby(['otaz', 'dtaz', 'deptm', 'arrtm']).count()[['id']].rename(columns={"id":"hov3d"})
    grouped_hov3_passenger = od_tazs[(od_tazs['mode']==5) & (od_tazs['dorp']==2)].groupby(['otaz', 'dtaz', 'deptm', 'arrtm']).count()[['id']].rename(columns={"id":"hov3p"})
    joined=grouped.join(grouped_hov2_driver, ['otaz', 'dtaz', 'deptm', 'arrtm']).join(grouped_hov2_passenger, ['otaz', 'dtaz', 'deptm', 'arrtm']).join(grouped_hov3_driver, ['otaz', 'dtaz', 'deptm', 'arrtm']).join(grouped_hov3_passenger, ['otaz', 'dtaz', 'deptm', 'arrtm']).fillna(0).sort_values('hov3p')
    
    joined['hov2']=(1+joined['hov2p'])/2-joined['hov2d']
    joined['hov2'] = joined['hov2'].apply(np.floor)

    #joined['final']=joined[['all', 'hov2d', 'hov2p', 'hov3d', 'hov3p']].apply(lambda x: x[0]-x[1]-x[2]+max(x[1], x[2])-x[3]-x[4]+max(x[3], int((1+x[4])/3)), axis=1)
    joined['demand']=joined[['all', 'hov2d', 'hov2p', 'hov3d', 'hov3p']].apply(lambda x: x[0]-x[2]-x[4], axis=1)
    joined = joined.reset_index()
    
    #new_od = joined.rename(columns={"otaz": "O", "dtaz": "D"})

    new_od["tt"] = new_od['arrtm'] - new_od['deptm']
    
    new_od["arrHour"] = new_od['arrtm'].apply(lambda x: int(x/30))
    return new_od


In [6]:
trips_path="C:\\Users\\Yiran\\Desktop\\sumo_calibration\\data_source\\"
dt_psrc = read_trips(trips_path)
# car: mode = [3,4,5]
mode = [1,6]
times = (0,24)
fields = ['id', 'opurp', 'dpurp', 'otaz', 'dtaz', 'mode', 'dorp', 'deptm', 'arrtm']
groups = ['otaz', 'dtaz', 'deptm', 'arrtm', 'mode', 'dorp']
group_names = {groups[i]:i for i in range(len(groups))}
od_tazs = dt_psrc[(dt_psrc['mode'].isin(mode)) & (dt_psrc['deptm']>=times[0]*60) & (dt_psrc['deptm']<times[1]*60)][fields]


In [7]:
psrc_ods=get_vols(od_tazs)
psrc_ods = psrc_ods[psrc_ods['O'].isin(downtown_tazs)].sort_values(by=['O', 'D', 'deptm']).reset_index()[0::10]

In [8]:
# Select in-network TAZs
#get information we need
#get taz file
pathfile = 'C:/Users/Yiran/Desktop/sumo_calibration/sumo_network_test'
os.chdir(pathfile)
datapath = '../Taz_bigger_Seattle_all_transit.add.xml'
tree = ET.parse(datapath)
root = tree.getroot()
selected_taz = []
for taz in root.findall('taz'):
    tazid = taz.get('id')
    selected_taz.append(tazid)
    
for i in range(len(selected_taz)):
    selected_taz[i] = int(selected_taz[i])


In [9]:
#conver dt_psrc_files
#select people opurp is home or is work
mode_keys = {1:'walk',2:'bike',3:'sov',4:'hov',5:'hov3+',6:'walk to transit',8:'school bus',0:'other'}
dt_psrc = dt_psrc[['otaz', 'dtaz', 'mode', 'opurp','dpurp']]

#dt_od = np.array(dt_psrc)
decode_mode = []
for mode_id in dt_psrc['mode']:
    #ped
    if mode_id == 1:
        decode_mode.append(1)
    #ped to transit
    elif mode_id == 6:
        decode_mode.append(2)
    #bike
    elif mode_id == 2:
        decode_mode.append(3)
    #veh
    elif mode_id in [3, 4, 5]:
        decode_mode.append(4)
    #other
    elif mode_id == 8:
        decode_mode.append(5)
    else:
        #print(mode_id)
        decode_mode.append(0)
        
dt_psrc['decode'] = decode_mode
dt_psrc.head(3)

,otaz,dtaz,mode,opurp,dpurp,decode
0,19,360,3,0,4,4
1,360,19,3,4,0,4
2,20,54,1,0,7,1


In [10]:
#assign value to specific od
def label_specific_od(dt_psrc, selected_taz, new_colname):
    ori = []
    for origin in dt_psrc['otaz']:
        if origin in selected_taz:
            ori.append(1)
        else:
            ori.append(-1)
    dest = []
    for destination in dt_psrc['dtaz']:
        if destination in selected_taz:
            dest.append(1)
        else:
            dest.append(-1)

    dt_psrc[new_colname[0]] = ori
    dt_psrc[new_colname[1]] = dest
    dt_psrc[new_colname[2]] = dt_psrc[new_colname[0]]+dt_psrc[new_colname[1]]
    return(dt_psrc)

def assign_to_taz(taz, assigned_taz, taz_map, sumo_taz):
    for i in taz:
        if i in sumo_taz:
            print('warning, taz already in sumo')
        else:
            if i not in taz_map:
                taz_map[i] = assigned_taz
    return(taz_map)

In [19]:
newcol = ['ori_id','dest_id','od']
dt_psrc = label_specific_od(dt_psrc, selected_taz, newcol)
dt_psrc = dt_psrc[dt_psrc['od']!=-2]

In [20]:
dt_psrc.head(3)

,otaz,dtaz,mode,opurp,dpurp,decode,ori_id,dest_id,od
40,586,584,3,6,5,4,-1,1,0
41,584,20,3,5,0,4,1,-1,0
79,19,426,6,0,4,2,-1,1,0


,TAZ,COUNTY_FIP,COUNTY_NM,geometry,lat,lon
0,1,033,King,"POLYGON ((-122.36096 47.72735, -122.36096 47.7...",47.726942,-122.392753
1,2,033,King,"POLYGON ((-122.35557 47.72505, -122.35556 47.7...",47.728688,-122.358265
2,3,033,King,"POLYGON ((-122.35559 47.73413, -122.35427 47.7...",47.731244,-122.350285


In [21]:
psrc_ods

NameError: name 'psrc_ods' is not defined

,index,O,D,deptm,arrtm,all,hov2d,hov2p,hov3d,hov3p,hov2,demand,tt,arrHour,cO,cD
0,7602486,426,2,902,924,1,0.0,0.0,0.0,0.0,0.0,1.0,22,30,426,5011
10,7602438,426,12,1012,1034,1,1.0,0.0,0.0,0.0,-1.0,1.0,22,34,426,5001
20,7602397,426,20,997,1021,1,0.0,0.0,0.0,0.0,0.0,1.0,24,34,426,5001
30,7602396,426,23,926,946,1,0.0,0.0,0.0,0.0,0.0,1.0,20,31,426,5001
40,7602366,426,31,529,548,1,0.0,0.0,0.0,0.0,0.0,1.0,19,18,426,5011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315780,7196632,653,2666,910,953,1,0.0,0.0,0.0,0.0,0.0,1.0,43,31,653,5001
315790,7196660,653,2971,947,1006,1,0.0,0.0,0.0,0.0,0.0,1.0,59,33,653,5003
315800,7196688,653,3148,806,845,1,0.0,0.0,0.0,0.0,0.0,1.0,39,28,653,5003
315810,7196697,653,3306,980,1057,1,0.0,0.0,0.0,0.0,0.0,1.0,77,35,653,5003


In [23]:
#psrc_ods_converted.to_csv("psrc_ods.csv")

,index,O,D,deptm,arrtm,all,hov2d,hov2p,hov3d,hov3p,hov2,demand,tt,arrHour,cO,cD
0,7602486,426,2,902,924,1,0.0,0.0,0.0,0.0,0.0,1.0,22,30,426,5011
10,7602438,426,12,1012,1034,1,1.0,0.0,0.0,0.0,-1.0,1.0,22,34,426,5001
20,7602397,426,20,997,1021,1,0.0,0.0,0.0,0.0,0.0,1.0,24,34,426,5001
30,7602396,426,23,926,946,1,0.0,0.0,0.0,0.0,0.0,1.0,20,31,426,5001
40,7602366,426,31,529,548,1,0.0,0.0,0.0,0.0,0.0,1.0,19,18,426,5011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315780,7196632,653,2666,910,953,1,0.0,0.0,0.0,0.0,0.0,1.0,43,31,653,5001
315790,7196660,653,2971,947,1006,1,0.0,0.0,0.0,0.0,0.0,1.0,59,33,653,5003
315800,7196688,653,3148,806,845,1,0.0,0.0,0.0,0.0,0.0,1.0,39,28,653,5003
315810,7196697,653,3306,980,1057,1,0.0,0.0,0.0,0.0,0.0,1.0,77,35,653,5003


In [25]:
v = p.copy()
v.loc[v['cO'].isin([5010])  & ((v['deptm']+v['arrtm'])%50<16), 'cO'] = 6001
v.loc[v['cO'].isin([5010])  & ((v['deptm']+v['arrtm'])%50<22), 'cO'] = 6002
v.loc[v['cO'].isin([5010])  & ((v['deptm']+v['arrtm'])%50<28), 'cO'] = 6003
v.loc[v['cO'].isin([5010])]

v.loc[v['cO'].isin([5000])  & ((v['deptm']+v['arrtm'])%112<4), 'cO'] = 6004
v.loc[v['cO'].isin([5000])  & ((v['deptm']+v['arrtm'])%112<42), 'cO'] = 6005
v.loc[v['cO'].isin([5000])  & ((v['deptm']+v['arrtm'])%112<45), 'cO'] = 6006
v.loc[v['cO'].isin([5000])]

,O,D,deptm,arrtm,all,hov2d,hov2p,hov3d,hov3p,hov2,demand,tt,arrHour,cO,cD
93,2432,520,353,408,1,0.0,0.0,0.0,0.0,0.0,1.0,55,13,5000,520
95,2432,543,404,474,1,0.0,0.0,0.0,0.0,0.0,1.0,70,15,5000,543
97,2432,566,405,464,1,0.0,0.0,1.0,0.0,0.0,1.0,59,15,5000,566
98,2432,650,337,391,1,0.0,0.0,0.0,0.0,0.0,1.0,54,13,5000,650
101,2432,609,353,398,1,0.0,0.0,1.0,0.0,0.0,1.0,45,13,5000,609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2088397,1731,529,475,512,1,1.0,0.0,0.0,0.0,-1.0,1.0,37,17,5000,529
2088420,1731,630,525,557,1,0.0,0.0,0.0,0.0,0.0,1.0,32,18,5000,630
2088421,1731,631,394,439,1,0.0,0.0,0.0,0.0,0.0,1.0,45,14,5000,631
2088424,1731,650,467,504,1,0.0,0.0,0.0,0.0,0.0,1.0,37,16,5000,650


In [26]:
v['depart'] = (v['deptm'])*60 + (v['O']+v['D']+v['arrtm'])%60
v.loc[v['cO']>=5000, 'depart']=(v['arrtm'])*60 -10*60- (v['O']+v['D']+v['deptm'])%60
xx = v.sort_values(by='depart')
xx

,O,D,deptm,arrtm,all,hov2d,hov2p,hov3d,hov3p,hov2,demand,tt,arrHour,cO,cD,depart
1686209,586,581,300,305,1,0.0,0.0,0.0,0.0,0.0,1.0,5,10,5006,581,17673
1800149,425,407,300,307,1,0.0,0.0,0.0,0.0,0.0,1.0,7,10,5006,427,17768
1795842,423,426,303,310,1,1.0,0.0,0.0,0.0,-1.0,1.0,7,10,5006,426,17988
1670885,561,573,301,311,1,0.0,0.0,0.0,0.0,0.0,1.0,10,10,5006,573,18005
1764762,454,443,300,309,1,0.0,0.0,0.0,0.0,0.0,1.0,9,10,454,443,18006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648325,3447,384,573,666,1,0.0,0.0,0.0,0.0,0.0,1.0,93,22,5002,434,39336
983444,2715,473,599,666,1,0.0,0.0,1.0,0.0,0.0,1.0,67,22,5002,473,39353
615172,3423,431,598,667,1,0.0,0.0,1.0,0.0,0.0,1.0,69,22,5002,431,39408
997492,2681,584,592,671,1,0.0,0.0,0.0,0.0,0.0,1.0,79,22,5002,584,39643


In [27]:
xx['hour'] = xx[['depart', 'cO', 'cD']].apply(lambda x: int(x[0]/3600), axis=1)
tst = xx[['depart', 'cO', 'cD', 'hour']].groupby(['hour', 'cO']).count().reset_index().sort_values(['cO', 'hour'])
tst[tst['cO']>=5000].head(50)

,hour,cO,depart,cD
159,5,5000,370,370
334,6,5000,2050,2050
512,7,5000,3739,3739
690,8,5000,4116,4116
869,9,5000,2619,2619
881,10,5000,636,636
160,5,5002,623,623
335,6,5002,2366,2366
513,7,5002,4017,4017
691,8,5002,4422,4422


In [33]:
filtered = xx[(xx['cO']>=5000) | (xx['cD']>=5000)]

In [63]:

#path = f'{base_path}\\sumo_network_test\\appsim\\veh_od_downtown_10pct.rou.xml'       
#write_xml_od(p, path)

NameError: name 'p' is not defined